HOLO everyone. 
Today we are going to make an image classifier to determine its HOLO or not.
Let's get cereal!

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [3]:
model = Sequential()

# First coat
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), data_format='channels_first'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(1, 1), data_format='channels_first'))

# Second coat
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format='channels_first'))

# Taco 
model.add(Flatten())  
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [4]:
batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'train',  
        target_size=(150, 150),  
        batch_size=batch_size,
        class_mode='binary')  


validation_generator = test_datagen.flow_from_directory(
        'validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

# time to bake
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
# bake that shit for a few minutes


model.save('holo_model.h5')

Found 229 images belonging to 2 classes.
Found 76 images belonging to 2 classes.
Epoch 1/10
125/125 [==============================] - 59s 471ms/step - loss: 0.7618 - acc: 0.5613 - val_loss: 0.6881 - val_acc: 0.5263
Epoch 2/10
125/125 [==============================] - 55s 440ms/step - loss: 0.6670 - acc: 0.6371 - val_loss: 0.6456 - val_acc: 0.6053
Epoch 3/10
125/125 [==============================] - 55s 441ms/step - loss: 0.5778 - acc: 0.7128 - val_loss: 1.1187 - val_acc: 0.5000
Epoch 4/10
125/125 [==============================] - 56s 446ms/step - loss: 0.5163 - acc: 0.7441 - val_loss: 0.6109 - val_acc: 0.6579
Epoch 5/10
125/125 [==============================] - 58s 464ms/step - loss: 0.4594 - acc: 0.7948 - val_loss: 0.6703 - val_acc: 0.6579
Epoch 6/10
125/125 [==============================] - 55s 442ms/step - loss: 0.4080 - acc: 0.8186 - val_loss: 0.6974 - val_acc: 0.7763
Epoch 7/10
125/125 [==============================] - 53s 424ms/step - loss: 0.3572 - acc: 0.8481 - val_loss:

In [7]:
from IPython.display import display, HTML
from keras.models import load_model
from PIL import Image
import numpy as np

model = load_model('holo_model.h5')
result_dict = {val:key for (key, val) in train_generator.class_indices.items()}

for i in range(1, 6):
    img = load_img('test/beauty_guru_{}.jpg'.format(i))

    resized_img = img.resize( (150, 150), Image.BILINEAR )
    resized_img = np.reshape(resized_img,[1,150,150,3])
    # whaddaya think
    classes = model.predict_classes(resized_img)
    
    result_text = "It's {0}".format(' '.join(result_dict[classes[0][0]].split('_')).upper())
    display(HTML('''<h1>Is it real HOLO ?</h1><img style="width:50%" src='test/beauty_guru_{0}.jpg'><h1>{1}</h1>'''.format(i, result_text)))